In [1]:
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [2]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [3]:
test_labels.shape

(10000,)

In [4]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
    
    
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv1_bn_relu = tf.nn.relu(conv1_bn)
    
    conv2 = tf.layers.conv2d(
      inputs=conv1_bn_relu,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)
    
    dropout1 = tf.layers.dropout(
      inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)


    
    
    conv3 = tf.layers.conv2d(
      inputs=dropout1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv3_bn_relu = tf.nn.relu(conv3_bn)
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn_relu,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout2 = tf.layers.dropout(
      inputs=pool2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    
    conv5 = tf.layers.conv2d(
      inputs=dropout2,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv5_bn_relu = tf.nn.relu(conv5_bn)
    
    conv6 = tf.layers.conv2d(
      inputs=conv5_bn_relu,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout3 = tf.layers.dropout(
      inputs=pool3, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    flattened = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    dense1 = tf.layers.dense(inputs=flattened, units=2048, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




/home/uttaran/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/uttaran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
print(train_labels.shape)
result = []
tf.logging.set_verbosity(tf.logging.ERROR)

(50000,)


In [6]:
# Evaluate the model and print results
def current_loss():
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    # output=list(predictions)
    #print("Train",eval_result)


    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    # output=list(predictions)
    #print("Test",eval_result)
    print (eval_result1)
    print (eval_result2)

In [7]:
# Create the Estimator
for i in range (0,1):
    cifar_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_6_conv_earlystop")


    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=128,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps=1000,
      hooks=None)
 
    print ("\n\nAfter " )
    print((i+1)*1000)
    print(" ")
    current_loss()
# print("eva",type(eval_data))




After 
1000
 
{'accuracy': 0.55968, 'loss': 1.232317, 'global_step': 1000}
{'accuracy': 0.5541, 'loss': 1.2561017, 'global_step': 1000}
